# 用 Map-Reduce 摘要長篇小說
在這個範例中，我們將示範如何使用 LangChain，對一份長文件進行 Map-Reduce 摘要。

## 建立環境與安裝套件

In [ ]:
# 安裝必要套件
!pip install langgraph openai

## 策略ㄧ、分而治之

### 第一步、建立環境與安裝套件

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import TokenTextSplitter

# 假設我們使用的是 GPT-3.5，其視窗較小
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 讀取你的文本文件
with open("./your_long_book.txt", "r", encoding='utf-8') as f:
  book_raw_data = f.read()

# 建立文本分割器
# chunk_size: 設定每個區塊的大小為 3000 tokens
# chunk_overlap: 設定區塊之間重疊 100 tokens，這就像翻頁時會多看一眼前一頁的最後一行，確保語意連貫
text_splitter = TokenTextSplitter(chunk_size=3000, chunk_overlap=100)
docs = text_splitter.create_documents([book_raw_data])

# 現在 `docs` 是一個包含多個小文本區塊的清單
print(f"文件被分割成了 {len(docs)} 個區塊。")

## 第二步：準備與分割文件

延續「台灣夜市文化」的例子，實際操作如何從維基百科、PDF 研究報告、Word 文件等多種來源，產出一份高品質的整合報告。

In [ ]:
# 建立 Map-Reduce 摘要鏈
# LangChain 會自動處理 Map 和 Reduce 兩個階段
map_reduce_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    # verbose=True 可以讓你看到每一步的詳細過程與 Prompt
    verbose=True
)

# 執行摘要鏈
final_summary = map_reduce_chain.invoke({"input_documents": docs})

print("\n---最終摘要---")
print(final_summary['output_text'])

## 策略二、層層迭代

### 第一步、載入不同來源的文件

In [ ]:
1.	from langchain_community.document_loaders import WikipediaLoader, PyPDFLoader, Docx2txtLoader
2.	
3.	# 1. 從維基百科載入資料
4.	wiki_loader = WikipediaLoader(query="台灣夜市", load_max_docs=1, lang="zh")
5.	wiki_docs = wiki_loader.load()
6.	
7.	# 2. 從本地 PDF 檔案載入
8.	pdf_loader = PyPDFLoader("./data/tourism_report.pdf")
9.	pdf_docs = pdf_loader.load()
10.	
11.	# 3. 從本地 Word 檔案載入
12.	word_loader = Docx2txtLoader("./data/cultural_essay.docx")
13.	word_docs = word_loader.load()
14.	
15.	# 將所有來源的文件整合成一個清單
16.	all_docs = wiki_docs + pdf_docs + word_docs
17.	print(f"總共載入了 {len(all_docs)} 份文件區塊。")


In [ ]:
### 第二步、定義 Refine Chain

In [ ]:
# 建立 Refine 摘要鏈
# LangChain 會自動處理初步摘要和後續的 Refine 步驟
refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    # return_intermediate_steps=True 可以讓我們看到每一次迭代中間產生的摘要
    return_intermediate_steps=True,
    verbose=True
)

# 執行摘要鏈
refine_summary = refine_chain.invoke({"input_documents": all_docs})

print("\n---每一次的迭代摘要---")
for i, step in enumerate(refine_summary["intermediate_steps"]):
    print(f"第 {i+1} 步摘要: {step}\n")

print("\n---最終完善摘要---")
print(refine_summary['output_text'])
